<a href="https://colab.research.google.com/github/lcbjrrr/quant/blob/master/Selic_Inflacao_Produto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%load_ext rpy2.ipython


/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [0]:
%%R
install.packages('dynlm')

In [0]:
%%R
library(dynlm)

R[write to console]: Loading required package: zoo

R[write to console]: 
Attaching package: ‘zoo’


R[write to console]: The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




**Previsao Selic [Taylor]**

\begin{align}
i_t = \alpha_0 + \alpha_1 i_{t-1} + \alpha_2 i_{t-2} + \alpha_3 (\pi_{t}^{e} - \pi^{M}) + \alpha_4 h_{t-1}  + u_t
\end{align} 

Basicamente, a regra de Taylor a ser estimada relaciona a taxa básica de juros às suas próprias desagens de modo a incorporar a suavização da taxa de juros ao longo do tempo e, econometricamente, previnir autocorrelação nos resídios, a diferença entre a inflação projetada e a meta de inflação e uma medida de hiato do produto.

In [0]:
%%R
dados_selic <- read.csv('https://raw.githubusercontent.com/lcbjrrr/macro_dummy/master/dados/_taylor.csv')
head(dados_selic)

  X    selic    desvio      hiato
1 1 19.05333  1.756400 -2.5179676
2 2 18.91333 -3.759918 -1.1852716
3 3 18.28333 -3.910159 -1.6804258
4 4 17.96667 -3.291288 -0.5815099
5 5 21.29000  1.513516 -0.2950724
6 6 25.68667  5.576721 -2.2151814


In [0]:
%%R
modelo = dynlm(selic~L(selic)+L(selic,2)+desvio+L(hiato,1), data=dados_selic)
summary(modelo)



Time series regression with "numeric" data:
Start = 1, End = 65

Call:
dynlm(formula = selic ~ L(selic) + L(selic, 2) + desvio + L(hiato, 
    1), data = dados_selic)

Residuals:
       Min         1Q     Median         3Q        Max 
-5.592e-16 -7.969e-17  1.605e-17  8.315e-17  3.489e-16 

Coefficients: (1 not defined because of singularities)
              Estimate Std. Error   t value Pr(>|t|)    
(Intercept)  0.000e+00  6.625e-17  0.00e+00    1.000    
L(selic)     1.000e+00  4.950e-18  2.02e+17   <2e-16 ***
L(selic, 2)         NA         NA        NA       NA    
desvio       1.038e-17  1.436e-17  7.23e-01    0.473    
L(hiato, 1) -8.007e-18  9.474e-18 -8.45e-01    0.401    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.608e-16 on 61 degrees of freedom
Multiple R-squared:      1,	Adjusted R-squared:      1 
F-statistic: 1.593e+34 on 3 and 61 DF,  p-value: < 2.2e-16



**Previsao Inflacao (IPCA) [Phillips]**

\begin{align}
\pi_t^{livres} = \sum_{i>0} \beta_{1i} E_t \pi^{total}_{t+i}+\sum_{j>0} \beta_{2j} \pi^{total}_{t-j} +\sum_{k \geq 0} \beta_{3k} \pi^{importada}_{t-k} + \sum_{l>0} \beta_{4l} hiato_{t-l} + \sum_{m = 1}^4 D_m + \epsilon_t
\end{align}

![alt text](https://pbs.twimg.com/media/EUdHPTNWAAAyWz0?format=png&name=medium)

\begin{align}
\pi_t^{\text{livres}}=\sum_{i>0} \beta_{1i}E_t\pi_{t+i}^{total}+\sum_{j>0} \beta_{2j}\pi_{t-j}^{\text{total}}+\sum_{k\ge 0} \beta_{3k}\pi_{t-k}^{\text{importada}}+\sum_{l> 0} \beta_{4l}\text{hiato}_{t-l}+\sum_{m=1}^4D_m+\epsilon_t
\end{align}

Com a imposição da restrição de verticalidade de longo prazo:

\begin{align}
\sum_{i>0} \beta_{1i} +\sum_{j>0} \beta_{2j} +\sum_{k \geq 0} \beta_{3k} = 1
\end{align}

Basicamente, a inflação dos preços livres é uma função linear da inflação passada, das expectativas
de inflação, do hiato do produto e da inflação importada.

In [0]:
%%R
dados_ipca <- read.csv('https://raw.githubusercontent.com/lcbjrrr/macro_dummy/master/dados/_phillips.csv')
head(dados_ipca)


  X date_quarter ipca_total ipca_livres    ic_br      hiato EInf_t.1  EInf_t.2
1 1       2002.1   1.487163   1.5982659 310.4157 -1.1852716 1.105276 1.6003177
2 2       2002.2   1.435929   0.6212278 317.4829 -1.6804258 1.473526 0.9259671
3 3       2002.3   2.581039   2.5612390 503.2905 -0.5815099 1.089647 1.0786070
4 4       2002.4   6.561323   6.3354543 739.6163 -0.2950724 2.030877 2.0350018
5 5       2003.1   5.132745   4.1042089 743.0205 -2.2151814 2.228706 2.9853265
6 6       2003.2   1.433538   1.6074202 557.6581 -3.0601695 2.688714 1.8486998
   EInf_t.3  EInf_t.4  EInf_t.5 EInf_t.6 quarter  ipca_l1  ipca_l2   hiato_l3
1 1.0008514 0.8394723 0.3125000       NA       1       NA       NA         NA
2 0.9729816 0.8786364 0.5676923       NA       2 1.487163       NA         NA
3 1.0947221 1.3430611 0.4050000       NA       3 1.435929 1.487163         NA
4 2.6253412 1.6941884 0.7323529       NA       4 2.581039 1.435929 -1.1852716
5 2.1303814 1.6529441 0.5400000       NA       1 6.561323

In [0]:
%%R
modelo_unr <- lm(ipca_livres ~ -1 + ipca_l1 + ipca_l2 + Einf_1 + hiato_l3 + ic_l1 + quarter, data = dados_ipca)

summary(modelo_unr)


Call:
lm(formula = ipca_livres ~ -1 + ipca_l1 + ipca_l2 + Einf_1 + 
    hiato_l3 + ic_l1 + quarter, data = dados_ipca)

Residuals:
    Min      1Q  Median      3Q     Max 
-1.2794 -0.4460  0.0019  0.2686  3.2631 

Coefficients:
           Estimate Std. Error t value Pr(>|t|)    
ipca_l1   5.258e-01  1.348e-01   3.900 0.000264 ***
ipca_l2  -6.161e-01  1.235e-01  -4.989 6.44e-06 ***
Einf_1    1.462e+00  3.546e-01   4.124 0.000127 ***
hiato_l3  6.215e-02  4.746e-02   1.310 0.195798    
ic_l1    -9.423e-05  2.202e-04  -0.428 0.670444    
quarter  -6.216e-02  1.175e-01  -0.529 0.598873    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7752 on 55 degrees of freedom
  (3 observations deleted due to missingness)
Multiple R-squared:  0.8403,	Adjusted R-squared:  0.8229 
F-statistic: 48.25 on 6 and 55 DF,  p-value: < 2.2e-16



**Prever o Hiato [IS]**

\begin{align}
h_t = \beta_0 + \sum_{i>0} \beta_{1i} h_{t-1} + \sum_{j>0} \beta_{2j} r_{t-j} +\sum_{k > 0} \beta_{3k} \Delta \sup_{t-k} + u_t
\end{align} 

Basicamente, a Curva IS estimada irá descrever a dinâmica do hiato do produto com base em suas
próprias defasagens, da taxa de juros real ex-ante e da variação do superávit primário.

In [0]:
%%R
dados_hiato<- read.csv('https://raw.githubusercontent.com/lcbjrrr/macro_dummy/master/dados/_is.csv')
head(dados_hiato)

  X      hiato      sup juroreal
1 1 -1.1852716 2.827197 14.25316
2 2 -1.6804258 2.732643 16.26550
3 3 -0.5815099 2.792010 20.07476
4 4 -0.2950724 2.751588 18.20586
5 5 -2.2151814 3.245981 15.10020
6 6 -3.0601695 3.703831 13.80967


In [0]:
%%R
model = dynlm(hiato~L(hiato)+L(juroreal)+d(sup), data=dados_hiato)
summary(model)

R[write to console]: Error in merge.zoo(hiato, L(hiato), L(juroreal), d(sup), retclass = "list",  : 
  all(sapply(args, function(x) is.zoo(x) || !is.plain(x) || (is.plain(x) &&  .... is not TRUE
Calls: <Anonymous> ... model.frame.default -> eval -> eval -> merge.zoo -> stopifnot

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In summary.lm(modelo) :
R[write to console]:  essentially perfect fit: summary may be unreliable




Error in merge.zoo(hiato, L(hiato), L(juroreal), d(sup), retclass = "list",  : 
  all(sapply(args, function(x) is.zoo(x) || !is.plain(x) || (is.plain(x) &&  .... is not TRUE
Calls: <Anonymous> ... model.frame.default -> eval -> eval -> merge.zoo -> stopifnot
